In [1]:
import numpy as np

from base import *

In [2]:
def print_estimate(models, prm_names):
    for name in prm_names:
        print(f"{'':<5}{name:<15}", end="")
    print()
    for prm, prm_err in models:
        for reg, reg_err in zip(prm, prm_err):
            for est, err in zip(
                reg, reg_err.diagonal()**0.5):
                print(f"{est:8.3g} ({err:8.3g})", end=" ")
            print()

In [3]:
seed = 179
rng = np.random.default_rng(seed)

In [130]:
n = 1001
d = 5

X = rng.standard_normal((d, n))
b = rng.uniform(0, 1, (1, d))
w = rng.uniform(0, 1, n) * 10

tries = 1000
summ = 0

for _ in range(tries):
    Y_err = rng.standard_normal((1, n))
    Y = b @ X + Y_err
    prm, prm_err = estimate_wls(Y, X, w, get_hc1_err)
    dev = prm - b
    summ += np.abs(dev[0][0]) < (prm_err[0].diagonal()[0])**0.5

print(summ / tries)

0.68


In [135]:
n = 1001
d = 5
r = 10

Z = rng.standard_normal((r, n))
g = rng.uniform(0, 1, (d, r))
b = rng.uniform(0, 1, (1, d))
b_err = rng.uniform(0, 1, (1, d))
w = rng.uniform(0, 1, n) * 10

tries = 1000
summ = 0

for _ in range(tries):
    X_err = rng.standard_normal((d, n))
    X = g @ Z + X_err
    Y_err = rng.standard_normal((1, n))
    Y = (b @ g) @ Z + b_err @ X_err + Y_err
    prm, prm_err, _, _ = estimate_wtsls(Y, X, Z, w, get_hc1_err)
    dev = prm - b
    summ += np.abs(dev[0][0]) < (prm_err[0].diagonal()[0])**0.5

print(summ / tries)

0.687


In [138]:
n = 1001
d = 5
r = 10

Z = rng.standard_normal((r, n))
g = rng.uniform(0, 1, (d, r))
b = rng.uniform(0, 1, (1, d))
b_err = rng.uniform(0, 1, (1, d))
w = rng.uniform(0, 1, n) * 10

X_err = rng.standard_normal((d, n))
X = g @ Z + X_err
Y_err = rng.standard_normal((1, n))
Y = (b @ g) @ Z + b_err @ X_err + Y_err

print_estimate(
    (
        estimate_ols(Y, X, get_hc0_err),
        estimate_ols(Y, X, get_hc1_err),
        estimate_ols(Y, X, get_hc2_err),
        estimate_wls(Y, X, w, get_hc1_err),
        estimate_wls(Y, X, w, get_hc2_err)),
    ('a', 'b', 'c', 'd', 'e'))
print()
print_estimate(
    (
        estimate_tsls(Y, X, Z, get_hc0_err)[:2],
        estimate_tsls(Y, X, Z, get_hc1_err)[:2],
        estimate_tsls(Y, X, Z, get_hc2_err)[:2],
        estimate_wtsls(Y, X, Z, w, get_hc1_err)[:2],
        estimate_wtsls(Y, X, Z, w, get_hc2_err)[:2]),
    ('a', 'b', 'c', 'd', 'e'))
b

     a                   b                   c                   d                   e              
    0.21 (  0.0268)    0.277 (  0.0251)    0.782 (  0.0258)    0.428 (  0.0245)    0.612 (  0.0244) 
    0.21 (  0.0268)    0.277 (  0.0252)    0.782 (  0.0258)    0.428 (  0.0245)    0.612 (  0.0245) 
    0.21 (  0.0269)    0.277 (  0.0252)    0.782 (  0.0259)    0.428 (  0.0245)    0.612 (  0.0245) 
   0.203 (  0.0319)    0.279 (  0.0296)    0.782 (  0.0277)    0.426 (  0.0283)     0.62 (  0.0275) 
   0.203 (   0.032)    0.279 (  0.0297)    0.782 (  0.0278)    0.426 (  0.0283)     0.62 (  0.0275) 

     a                   b                   c                   d                   e              
   0.473 (  0.0703)    0.144 (  0.0493)     0.74 (  0.0493)    0.831 (   0.052)    0.136 (  0.0784) 
   0.473 (  0.0723)    0.144 (  0.0508)     0.74 (  0.0508)    0.831 (  0.0535)    0.136 (  0.0806) 
   0.473 (  0.0706)    0.144 (  0.0495)     0.74 (  0.0495)    0.831 (  0.0522)    0.136 (

array([[0.50677314, 0.16415471, 0.72815813, 0.85811569, 0.0814514 ]])